In [1]:
%matplotlib inline
import math

import numpy as np
import time
import matplotlib.pyplot as plt

from tools.corner_response_function import compute_gaussian_grad,compute_corner_response
from tools.visualization import reshape_as_images, imshow
from tools.data_loading import load_images
from tools.discretization import discretize_orientation, pin_as_vect
from tools.quantization import kmeans, vf_vector

In [2]:
filter_size = 3

In [3]:
X = load_images(type="train")
X = reshape_as_images(X) # 5000 x 32 x 32 x 3
images = X.mean(axis=3) # 5000 x 32 x 32
window_size = 5
stride = 3
patch_size = 5
pins = list()

R_size = (images.shape[1] - window_size - filter_size)//stride + 1
R = np.zeros((images.shape[0], R_size, R_size))

In [ ]:
for image_idx in range(images.shape[0]):
        image_mat = images[image_idx]
        im_size = image_mat.shape[0]  # normally 32
        image_grad_x, image_grad_y = compute_gaussian_grad(image_mat)

        for i in range(R_size):
            for j in range(R_size):
                I_x = image_grad_x[i*stride:i*stride+window_size,
                                   j*stride:j*stride+window_size]
                I_y = image_grad_y[i*stride:i*stride+window_size,
                                   j*stride:j*stride+window_size]

                R[image_idx, i, j] = compute_corner_response(I_x, I_y)
        # thresholding of R
        R[image_idx] = np.abs(R[image_idx])

In [ ]:
def from_R_to_im(x, y, window_size, stride):
    x = x * stride + (window_size - 1)//2 + (filter_size - 1)//2
    y = y * stride + (window_size - 1)//2 + (filter_size - 1)//2

    return (x, y)


def R_to_heatmap(R, window_size, stride, im_size):
    R_size = R.shape[0]
    # create heatmap
    heatmap = np.zeros((im_size, im_size))
    for i in range(R_size):
        for j in range(R_size):
            ix, iy = from_R_to_im(i, j, window_size, stride)
            heatmap[ix:ix+window_size,
                    iy:iy+window_size] = R[i, j]
    heatmap /= heatmap.max()
    return heatmap

In [ ]:
ind = 0

In [ ]:
i, j = np.where(R[ind] > np.percentile(R[ind], 75))
i, j = from_R_to_im(i, j, window_size, stride)

x, y = j, i

heatmap = R_to_heatmap(R[ind], window_size, stride, im_size)
imshow(images[ind], points_of_interest=(x, y), heatmap=heatmap)
ind +=1

In [ ]:
pins_img_ids = []
pins_coords = []
im_to_pins = dict.fromkeys(range(images.shape[0]), [])
for image_idx in range(images.shape[0]):
    i_s, j_s = np.where(
        R[image_idx] > np.percentile(R[image_idx], 75))
    i_s, j_s = from_R_to_im(i_s, j_s, window_size, stride)
    for i, j in zip(i_s, j_s):  # i, j are the coordinates in R of POI
        patch_x = image_grad_x[i-patch_size//2:i+patch_size//2+1,
                               j-patch_size//2:j+patch_size//2+1]
        patch_y = image_grad_y[i-patch_size//2:i+patch_size//2+1,
                               j-patch_size//2:j+patch_size//2+1]
        pin_as_matrix = discretize_orientation(patch_x, patch_y)
        pin = pin_as_vect(pin_as_matrix)
        pins.append(pin) # pins is list of all pins for all images
        # for visualization purposes
        pins_img_ids.append(image_idx)
        pins_coords.append((i,j))
        im_to_pins[image_idx].append(pin)

# Dataset separation

In [ ]:
sep_indices = np.random.permutation(X.shape[0])
training_idx, test_idx = sep_indices[:int(0.9*n_train)], sep_indices[int(0.9*n_train):]

In [ ]:
train_pins = list() #  list of all pins for training images
for idx in training_idx:
    train_pins += im_to_pins[idx]
pins_mat = np.vstack(train_pins)

In [ ]:
visual_features = kmeans(pins_mat, 66)

In [ ]:
vf_vector(train_pins, visual_features)

In [ ]:
y = np.argmin(np.linalg.norm(pins_mat[:, None, :] - visual_features, axis=2), axis=1)


nb_features = 3
for i in range(nb_features): # first three visual features
    indices = np.where(y == i)[0][:9]
    f, axarr = plt.subplots(3, 3)
    
    for aa in range(9):
        r = aa // 3
        s = aa - r * 3
        ii, jj = pins_coords[indices[aa]]
        image = X_sample[pins_img_ids[indices[aa]]][ii-patch_size//2:ii+patch_size//2+1,
                               jj-patch_size//2:jj+patch_size//2+1]
        image -= X_sample[pins_img_ids].min()
        image /= X_sample[pins_img_ids].max()
        axarr[r,s].imshow(image, interpolation="nearest")

# Actual pipeline

In [ ]:
Y_labels_train = load_labels()
Y_train = dummy_code(Y_labels_train)
n_classes = Y_train.shape[1]

X_proc = process_images(X, proc_type="bovf", centroids=centroids, im_to_pins=im_to_pins)
n_train, n_var = X_proc.shape
X_sample = X_proc[training_idx, :]
n_sample = X_sample.shape[0]
X_test = X_proc[test_idx, :]
n_test = X_test.shape[0]
Y_sample = Y_train[training_idx, :]

In [ ]:
# Training
kernel_type = "linear"
K_sample = kernel_matrix(X_sample, kernel_type=kernel_type)

classifier_type = "linear regression"
alpha = np.zeros((n_classes, n_sample))
for dig in range(n_classes):
    alpha[dig, :] = find_f(K_sample, Y_sample[:, dig],
                           prob_type=classifier_type, lamb=1.0)

# Evaluation
Y_pred = np.zeros((X_test.shape[0], n_classes))
for dig in range(n_classes):
    Y_pred[:, dig] = pred(X_sample, X_test, alpha[dig, :],
                          kernel_type=kernel_type)


Y_labels_pred = np.argmax(Y_pred, axis=1)
prec = np.mean(Y_labels_pred == Y_labels_train[test_idx])
print(prec)